In [ ]:
# Crop Each Band

# When you need to crop and stack a set of images, it is most efficient to first crop each image, and then stack it.
# es.crop_all() is an efficient way to crop all bands in an image quickly. The function will write out cropped rasters
# to a directory and return a list of file paths that can then be used with es.stack().

# the following syntax can be used

band_paths_list = es.crop_all(
    stack_band_paths, output_dir, crop_bound_utm13N, overwrite=True
)

# band_paths_list is a list of all filepaths created by the crop_all function
# stack_band_paths is a list of all filepaths to be cropped
# output_dir (string) is the output directory in which the cropped files will be returned; all images 
# produced will have the _crop suffix
# crop_bound is a geojson object of the bounds of the area of interest - as always, note that they must be in the same CRS
# overwrite will force overwrite any existing cropped files

In [ ]:
# es.crop_image() can be used for single image cropping - it takes in a rasterio object and crops it to a specified geojson object